In [498]:

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [499]:
train_data = pd.read_csv('../input/inputdataset/train.csv')
test_data = pd.read_csv('../input/inputdataset/test.csv')

train_data.head(3)
len(train_data)
len(test_data)

In [500]:
test_data.head()

In [501]:
#Data types
pd.DataFrame({"Data type":train_data.dtypes})

In [502]:

train_data.describe()

for i in train_data['cut'] :
    if i=='Fair':
        i="1"
    
    
train_data['cut'].head(30)


In [503]:
#Missing data
pd.DataFrame({"Missing values (%)": train_data.isnull().sum()/len(train_data.index)*100})

In [509]:
import gc

train_data['tst']=0
test_data['tst']=1
train_data = pd.concat([test_data, train_data], axis=0, copy=True)
len(train_data)
train_data.tail()





In [510]:

y = train_data.price


In [511]:
print("Unique values in 'cut' column in training data:", train_data['cut'].unique())
print("\nUnique values in 'cut' column in validation data:", train_data['clarity'].unique())

In [512]:
# All categorical columns
object_cols = [col for col in train_data.columns if train_data[col].dtype == "object"]

# Columns that can be safely label encoded
good_label_cols = [col for col in object_cols if 
                   set(train_data[col]) == set(train_data[col])]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

In [513]:
train_data.head()

In [514]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: train_data[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

In [515]:

sns.barplot(x=train_data.cut, y=y)


In [516]:
sns.barplot(x=train_data.color, y=y)

In [517]:

sns.barplot(x=train_data.clarity, y=y)

In [518]:

fig, axs = plt.subplots(nrows=3, figsize=(18,20))
# Plot histogram
sns.boxplot(train_data["table"], ax=axs[0])
sns.boxplot(train_data["depth"],ax=axs[1])
sns.boxplot(train_data["carat"], ax=axs[2])
# Remove scientific notation
axs[0].ticklabel_format(style='plain', axis='x')
axs[1].ticklabel_format(style='plain', axis='x')
axs[2].ticklabel_format(style='plain', axis='x')
plt.show()


In [519]:
# Histogram 
sns.distplot(a=train_data['depth'], kde=False)

In [520]:
sns.distplot(a=train_data['table'], kde=False)

In [521]:
sns.distplot(a=train_data['carat'], kde=False)

In [522]:
sns.distplot(a=train_data['z'], kde=False)

In [523]:
fig, axs = plt.subplots(nrows=3, figsize=(18,50))
sns.distplot((train_data['table'].dropna()), ax=axs[0])
sns.distplot((train_data['carat'].dropna()), ax=axs[1])
sns.distplot((train_data['depth'].dropna()), ax=axs[2])
plt.show()

In [525]:
from sklearn.model_selection import train_test_split
#separate target from predictors
#train_data.dropna(axis=0, subset=['price'], inplace=True)
y = train_data.price
train_data.drop(['price'], axis=1, inplace=True)
a= ['carat', 'depth', 'table']
X = train_data[a]
# Separate data into training and validation sets
#X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [526]:
#from xgboost import XGBRegressor
#from sklearn.metrics import mean_absolute_error

#my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
#my_model.fit(X_train, y_train, 
#            early_stopping_rounds=5, 
#             eval_set=[(X_valid, y_valid)], 
#             verbose=False)

#predictions = my_model.predict(X_valid)
#print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))


In [527]:
out=[]
def Zscore_outlier(df):
    m = np.mean(df)
    sd = np.std(df)
    for i in df: 
        z = (i-m)/sd
        if np.abs(z) > 3: 
            out.append(i)
    print("Outliers:",out)
Zscore_outlier(train_data['carat'])



In [528]:
def replace_outliers_z_score(dataframe, column, Z=3):

#Replace outliers with the mean values using the Z score

    from scipy.stats import zscore
    df = dataframe.copy(deep=True)
    df.dropna(inplace=True, subset=[column])
    # Calculate mean without outliers
    df["zscore"] = zscore(df[column])
    mean_ = df[(df["zscore"] > -Z) & (df["zscore"] < Z)][column].mean()
    # Replace with mean values
    dataframe[column] = dataframe[column].fillna(mean_)
    dataframe["zscore"] = zscore(dataframe[column])
    no_outliers = dataframe[(dataframe["zscore"] < -Z) | (dataframe["zscore"] > Z)].shape[0]
    dataframe.loc[(dataframe["zscore"] < -Z) | (dataframe["zscore"] > Z),column] = mean_
    # Print message
    print("Replaced:", no_outliers, " outliers in ", column)
    return dataframe.drop(columns="zscore")

In [529]:
#freatures=['carat','table']
#for c in train_data.columns:
#    if c != "id":
#        features = replace_outliers_z_score(train_data[features],c)

In [530]:
train_data.reset_index(drop=True, inplace=True)

In [531]:
def _find_outliers_iqr(dataframe, column):

#Find outliers using the 1.5*IQR rule.


    col = sorted(dataframe[column])
    q1, q3= np.percentile(col,[25,75])
    iqr = q3 - q1
    lower_bound = q1 -(1.5 * iqr)
    upper_bound = q3 +(1.5 * iqr)
    results = {"iqr": iqr, "lower_bound":lower_bound, "upper_bound":upper_bound}
    return results

In [532]:
def remove_outliers_iqr(dataframe, column):

#Remove outliers using the 1.5*IQR rule.


    outliers = _find_outliers_iqr(dataframe, column)
    removed = dataframe[(dataframe[column] < outliers["lower_bound"]) |
    (dataframe[column] > outliers["upper_bound"])].shape
    dataframe = dataframe[(dataframe[column] > outliers["lower_bound"]) &
    (dataframe[column] < outliers["upper_bound"])]
    print("Removed:", removed[0], " outliers")
    return dataframe

In [533]:
def remove_outliers_z_score(dataframe, column, Z=3):

#Remove outliers using the Z score. Values with more than 3 are removed.


    from scipy.stats import zscore
    dataframe["zscore"] = zscore(dataframe[column])
    removed = dataframe[(dataframe["zscore"] < -Z) |
    (dataframe["zscore"] > Z)].shape
    dataframe = dataframe[(dataframe["zscore"] > -Z) &
    (dataframe["zscore"] < Z)]
    print("Removed:", removed[0], " outliers of ", column)
    return dataframe.drop(columns="zscore")

In [534]:
def replace_outliers_z_score(dataframe, column, Z=3):

#Replace outliers with the mean values using the Z score.

    from scipy.stats import zscore
    df = dataframe.copy(deep=True)
    df.dropna(inplace=True, subset=[column])
    # Calculate mean without outliers
    df["zscore"] = zscore(df[column])
    mean_ = df[(df["zscore"] > -Z) & (df["zscore"] < Z)][column].mean()
    # Replace with mean values
    no_outliers = dataframe[column].isnull().sum()
    dataframe[column] = dataframe[column].fillna(mean_)
    dataframe["zscore"] = zscore(dataframe[column])
    dataframe.loc[(dataframe["zscore"] < -Z) | (dataframe["zscore"] > Z),column] = mean_
    # Print message
    print("Replaced:", no_outliers, " outliers in ", column)
    return dataframe.drop(columns="zscore")

In [535]:
train_data = replace_outliers_z_score(train_data,"table")
train_data = replace_outliers_z_score(train_data,"carat")
train_data = replace_outliers_z_score(train_data,"depth")

In [536]:
train_data.reset_index(drop=True, inplace=True)

In [537]:
fig, axs = plt.subplots(nrows=3, figsize=(18,20))
# Plot histogram
sns.boxplot(train_data["table"], ax=axs[0])
sns.boxplot(train_data["depth"],ax=axs[1])
sns.boxplot(train_data["carat"], ax=axs[2])
# Remove scientific notation
axs[0].ticklabel_format(style='plain', axis='x')
axs[1].ticklabel_format(style='plain', axis='x')
axs[2].ticklabel_format(style='plain', axis='x')
plt.show()

In [538]:
train = train_data[train_data['tst']==0]
test_data = train_data[train_data['tst']==1]
len(test_data)
#del train_data
y=y.dropna()




In [539]:
train=train.drop(columns=["tst"])
test_data=test_data.drop(columns=["tst"])

#len(y)
train_data=train.copy()



In [540]:
from sklearn.model_selection import train_test_split
#train_data.drop(['cut'], axis=1, inplace=True)
#train_data.drop(['clarity'], axis=1, inplace=True)
#train_data.drop(['color'], axis=1, inplace=True)
#train_data.drop(['x'], axis=1, inplace=True)
#train_data.drop(['y'], axis=1, inplace=True)
#train_data.drop(['z'], axis=1, inplace=True)
#y = train_data.price

# Break off validation set from training data
X_train, X_valid, y_train, y_valid = train_test_split(train_data, y,
                                                      train_size=0.8, test_size=0.2
                                                      )
# Separate data into training and validation sets

In [541]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

In [542]:
# All categorical columns
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Columns that can be safely label encoded
good_label_cols = [col for col in object_cols if 
                   set(X_train[col]) == set(X_valid[col])]
        
# Problematic columns that will be dropped from the dataset
bad_label_cols = list(set(object_cols)-set(good_label_cols))
        
print('Categorical columns that will be label encoded:', good_label_cols)
print('\nCategorical columns that will be dropped from the dataset:', bad_label_cols)

from sklearn.preprocessing import LabelEncoder

# Drop categorical columns that will not be encoded
label_X_train = X_train.drop(bad_label_cols, axis=1)
label_X_valid = X_valid.drop(bad_label_cols, axis=1)

# Apply label encoder 
label_encoder = LabelEncoder()

for col in set(good_label_cols):
    label_X_train[col] = label_encoder.fit_transform(X_train[col])
    label_X_valid[col] = label_encoder.transform(X_valid[col])  
    

print("MAE from Approach 2 (Label Encoding):") 
print(score_dataset(label_X_train, label_X_valid, y_train, y_valid))

In [543]:
###For test_data

# All categorical columns
object_cols = [col for col in test_data.columns if test_data[col].dtype == "object"]

# Columns that can be safely label encoded
good_label_cols = ['cut', 'color', 'clarity']


from sklearn.preprocessing import LabelEncoder

# Apply label encoder 
label_encoder = LabelEncoder()

for col in set(good_label_cols):
    test_data[col] = label_encoder.fit_transform(test_data[col])

In [544]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

# Select subset of predictors
#a= ['carat', 'depth', 'table']
#X_train = train_data[a]

my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)

my_model.fit(label_X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(label_X_valid, y_valid)], 
             verbose=False)

predictions = my_model.predict(label_X_valid)
#print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))



In [545]:

correlation = label_X_train.corr()
# Plot correlation
plt.figure(figsize=(20,18))
sns.heatmap(correlation, xticklabels=correlation.columns.values,
yticklabels=correlation.columns.values, annot = True, annot_kws={'size':10})
# Axis ticks size
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [547]:

test_preds = my_model.predict(test_data)
#predictions = my_model.predict(label_X_valid)
# The lines below shows how to save predictions in format used for competition scoring


output = pd.DataFrame({'id': test_data.id,'price': test_preds})
output.to_csv('sample_submission.csv', index=False)